In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd

In [2]:
def init_weights(shape):
    return tf.Variable(tf.random_normal(shape, stddev=0.01))

In [3]:
def model(X, w_h, w_o):
    h = tf.nn.sigmoid(tf.matmul(X, w_h))
    return tf.matmul(h, w_o)

In [4]:
X = tf.placeholder('float', [None, 52])
Y = tf.placeholder('float', [None, 2])

w_h = init_weights([52, 40])
w_o = init_weights([40, 2])

In [5]:
py_x = model(X, w_h, w_o)

In [6]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(py_x, Y))
train_op = tf.train.GradientDescentOptimizer(0.05).minimize(cost)
predict_op = tf.argmax(py_x, 1)

In [7]:
def LeaveOneOut(data1, data2, columnName, useLOO=False):
    grpOutcomes = data1.groupby(columnName).mean().reset_index()
    outcomes = data2['outcome'].values
    x = pd.merge(data2[[columnName, 'outcome']], grpOutcomes,
                 suffixes=('x_', ''),
                 how='left',
                 on=columnName,
                 left_index=True)['outcome']
    if(useLOO):
        x = ((x*x.shape[0])-outcomes)/(x.shape[0]-1)
    return x.fillna(x.mean())

In [8]:
directory = 'data/'
train = pd.read_csv(directory+'act_train.csv')
people = pd.read_csv(directory+'people.csv')

In [9]:
train = pd.merge(train, people,how='left',on='people_id',left_index=True)
train.fillna('-999', inplace=True)

In [10]:
features = train.columns.tolist()
features.remove('activity_id')
features.remove('people_id')
lootrain = pd.DataFrame()
for col in features:
    if(col != 'outcome'):
        print(col)
        lootrain[col] = LeaveOneOut(train, train, col, True).values

date_x
activity_category
char_1_x
char_2_x
char_3_x
char_4_x
char_5_x
char_6_x
char_7_x
char_8_x
char_9_x
char_10_x
char_1_y
group_1
char_2_y
date_y
char_3_y
char_4_y
char_5_y
char_6_y
char_7_y
char_8_y
char_9_y
char_10_y
char_11
char_12
char_13
char_14
char_15
char_16
char_17
char_18
char_19
char_20
char_21
char_22
char_23
char_24
char_25
char_26
char_27
char_28
char_29
char_30
char_31
char_32
char_33
char_34
char_35
char_36
char_37
char_38


In [11]:
lootrain.head()

,date_x,activity_category,char_1_x,char_2_x,char_3_x,char_4_x,char_5_x,char_6_x,char_7_x,char_8_x,...,char_29,char_30,char_31,char_32,char_33,char_34,char_35,char_36,char_37,char_38
0,0.443469,0.489206,0.446476,0.446476,0.446476,0.446476,0.446476,0.446476,0.446476,0.446476,...,0.402735,0.650801,0.652109,0.360108,0.388654,0.658206,0.65678,0.668789,0.356352,0.0
1,0.281099,0.510324,0.446476,0.446476,0.446476,0.446476,0.446476,0.446476,0.446476,0.446476,...,0.402735,0.650801,0.652109,0.360108,0.388654,0.658206,0.65678,0.668789,0.356352,0.0
2,0.281099,0.510324,0.446476,0.446476,0.446476,0.446476,0.446476,0.446476,0.446476,0.446476,...,0.402735,0.650801,0.652109,0.360108,0.388654,0.658206,0.65678,0.668789,0.356352,0.0
3,0.442657,0.510324,0.446476,0.446476,0.446476,0.446476,0.446476,0.446476,0.446476,0.446476,...,0.402735,0.650801,0.652109,0.360108,0.388654,0.658206,0.65678,0.668789,0.356352,0.0
4,0.443469,0.510324,0.446476,0.446476,0.446476,0.446476,0.446476,0.446476,0.446476,0.446476,...,0.402735,0.650801,0.652109,0.360108,0.388654,0.658206,0.65678,0.668789,0.356352,0.0


In [12]:
y_train = map(lambda x: [1,0] if x==0 else [0,1], train.outcome.as_matrix())
y_train = np.matrix(y_train)
label = train['outcome'].values

In [13]:
from sklearn.metrics import roc_auc_score

In [14]:
with tf.Session() as sess:
    tf.initialize_all_variables().run()
    
    
    for i in range(50):

        for start, end in zip(range(0, len(train), 500), range(500, len(train)+1,500)):
            sess.run(train_op, feed_dict={X: lootrain[start:end], Y: y_train[start:end]})
        predict = sess.run(predict_op, feed_dict={X: lootrain, Y: y_train})
        print(i, np.mean(label == predict))
        print(i, roc_auc_score(label, predict))
        
        
        #print(i, sess.run(predict_op, feed_dict={X: train, Y: y_train}) )
    


(0, 0.97038216603991001)
(0, 0.97099220771670047)
(1, 0.97051915290236934)
(1, 0.97101195896388348)
(2, 0.97062382724909901)
(2, 0.97108521121083735)
(3, 0.97066979294048894)
(3, 0.9711215842765597)
(4, 0.97065750508239468)
(4, 0.97110030578076834)
(5, 0.97068663185713677)
(5, 0.97111885072908677)
(6, 0.97071484842016831)
(6, 0.97114773630333118)
(7, 0.97067434399904251)
(7, 0.97109777888319382)
(8, 0.97062564767252035)
(8, 0.97105016779105624)
(9, 0.9706274680959418)
(9, 0.97104973822923324)
(10, 0.97063110894278459)
(10, 0.97104577935711334)


KeyboardInterrupt: 